In [1]:
from aicsimageio import AICSImage
from tifffile import imsave
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from itkwidgets import view 
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
plt.rcParams["figure.figsize"] = [16, 12]

# dots
from aicssegmentation.core.seg_dot import dot_3d, dot_3d_wrapper 
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from skimage.morphology import remove_small_objects, watershed, dilation, erosion, ball     # function for post-processing (size filter)
from skimage.feature import peak_local_max
from skimage.measure import label
from scipy.ndimage import distance_transform_edt

In [2]:
filename = r"\\allen\aics\assay-dev\users\Sandi\mFISH\notebooks\TOP2A_007_002_bc5.tiff"
reader = AICSImage(filename) 
IMG = reader.data.astype(np.float32)

print(IMG.shape)

Failed to parse XML for the provided file. Error: not well-formed (invalid token): line 1, column 0


(1, 1, 51, 2048, 2048)


In [3]:
# N_CHANNELS = IMG.shape[1] # N_CHANNELS = IMG.shape[1] 
# MID_SLICE = np.int(0.5*IMG.shape[2])

# fig, ax = plt.subplots(1, N_CHANNELS, figsize=(18,16), dpi=72, facecolor='w', edgecolor='k')
# if N_CHANNELS==1:
#     ax.axis('off')
#     ax.imshow(IMG[0,0,MID_SLICE,:,:], cmap=plt.cm.gray)
# else:
#     for channel in range(N_CHANNELS):
#         ax[channel].axis('off')
#         ax[channel].imshow(IMG[0,channel,MID_SLICE,:,:], cmap=plt.cm.gray)

In [4]:
IMG.shape

(1, 1, 51, 2048, 2048)

In [5]:
#####################
structure_channel = 0
#####################

struct_img0 = IMG[0,structure_channel,:,:,:].copy()
view(single_fluorescent_view(struct_img0))

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

<h2> Pre-Processing <h2> 

In [6]:
################################
intensity_scaling_param = [5000] # original: 8000 current: 5000

gaussian_smoothing_sigma = 1 
################################

# intensity normalization
struct_img = intensity_normalization(struct_img0, scaling_param=intensity_scaling_param)

# smoothing with gaussian filter
structure_img_smooth = image_smoothing_gaussian_slice_by_slice(struct_img, sigma=gaussian_smoothing_sigma)

intensity norm: min-max norm with upper bound 5000


<h2> Core Algorithm <h2>

In [7]:
################################
## PARAMETERS for this step ##
s3_param = [[0.9, 0.013]] # v1: [[0.7, 0.015]] v2: [[0.7, 0.03]] v3: [[0.7, 0.025]] v4: [[0.7, 0.015]] v5: [[0.9, 0.013]]
################################

bw = dot_3d_wrapper(structure_img_smooth, s3_param)

In [8]:
# view the segmentation result
viewer_bw = view(segmentation_quick_view(bw))
viewer_bw

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

<h2> Post-Processing <h2>

In [54]:
################################
## PARAMETERS for this step ## (v5)
minArea = 5 #v5: 5
################################

seg = remove_small_objects(bw>0, min_size=minArea, connectivity=1, in_place=False)

In [55]:
## save segmentation 
out=seg.astype(np.uint8)
out[out>0]=255

imsave("TOP2A_007_002_segbc5v5.tiff", out) 